## Nur eine Typklasse zum aufwärmen

Monoid (Körper, nur klein) => $(\mathbb{M}, \cdot, 0)$

Gesetze für ein Monoid: 

$1\cdot x = x; x \cdot 1 = x; (x \cdot y) \cdot z = x (\cdot y \cdot z)$

In [11]:
class Monoid m where
    mempty :: m
    mappend :: m -> m -> m

instance Monoid Integer where
    mempty = 0
    mappend = (+)

instance Monoid Bool where
    mempty = True
    mappend = (&&)

## IO - interagieren über in und out
- print und so ist eigenltich nur ein Seiteneffekt, zb gibt in python print("hallo") = None zurück
- und das ist ein problem mit dem lazy prinzip, wieso ausprinten wenn ich gleich None zurückgeben kann?
### Anwendung
1. Phase: Benutzer gibt Daten ein
2. Phase: Benutzer fragt Daten an

das funktioniert nicht nacheinander, weil haskell lazy ist

Teil1 der Lösung: IO aktionen sind Werte, diese werden zu einer gesamtaktion zusammengesetzt. Die wird dann ausgeführt, wenn sie ganz oben (in der main Funktion) ankommt
IO() ist ne "IO Unit"
```haskell
putStr :: String -> IO ()
putStrLn :: String -> IO ()
print :: Show a => a -> IO ()
nl :: IO ()
(>>) :: IO () -> IO () -> IO ()
```

In [12]:
main = putStr "Hallo " >>
        putStrLn "Welt" >>
        print 42

main

Hallo Welt
42

In [13]:
main2 = let ac1 = putStr "Hallo " in
        ac1 >>
        putStrLn "Welt" >>
        ac1 >>
        print 42

main2

Hallo Welt
Hallo 42

---

Das ist dasselbe
```haskell
let x = 3 in x + 2
x = 2 where x = 3
```

---

```haskell
let x = getLine in print x
-- DAS GEHT NICHT
```

```haskell
getLine >>= putStrLn 
```
Wenn ich das benutzen will, dann muss ich das in einer funktion zwischenspeichern
```haskell
getLine >>= (\t -> putStrLn t >> putStrLn t)
main3 = getLine >>= \t1 -> putStrLn t1 >>
        getLine >>= \t2 -> putStrLn t2 >>
        if t1 == t1 then putStrLn "gleich" else putStrLn "ungleich" >>
        putStrLn t1 ++ t2
```

In [14]:
-- getChar :: IO Char
getLine' :: IO String
getLine' = getChar >>= \c -> 
        if c == '\n' then return ""
        else getLine' >>= \cs -> return (c:cs)

main3 in python:
```python
def main3():
    t1 = input()
    print(t1)
    t2 = input()
    ...
```

In [15]:
main = do
    t1 <- getLine
    putStrLn t1
    t2 <- getLine
    putStrLn t2
    

für die Prüfung den Bind operator merken:
```haskell
(>>=) :: IO a -> (a -> IO b) -> IO b
```

## Mehrere Möglichkeiten des Binds
```haskell
-- 1.
return x >>= f = f x
-- mit:
x :: b
return x :: IO b
f :: (b -> IO a)
-- 2. 
action >>= return = action
-- mit
action :: IO a
action >>= return :: IO a
return :: (a -> IO a)
-- 3.
(act >>= f) >>>= g = act >>= (\res -> f res >>= g)
```


## Alle Datentypen, für die es eine map-funktion gibt
Funktor ist keine typklasse, sonder eine Typ konstruktor klasse
dh alle instanzen davon sind keine Typen, sondern Konstruktoren mit Stelligkeit 1

- Typ: `Int, Float, (), [Int], [[(Int, (), Bool)]]` alles `Kind *`
- Typkonstruktor mit Stelligkeit 1: `[], Maybe, Tree, Either` alles `Kind * -> *`
- Typkonstruktor mit Stelligkeit 2: `Either, (->), (,)` mit `Kind * -> * -> *`
- `Either Int` ist eine partielle Applikation des zweistelligen Typkonstruktors Either auf den Typ Int und hat Kind * -> *

- Typklassen abstrahieren über Typen: `Eq, Num, Show`
- Typkonstruktorklassen abstrahieren über Typkonstruktoren einer bestimmten Stelligkeit (oder partielle Applikationen hörerer Stelligkeit)

In [16]:
class Functor f where
    fmap :: (a -> b) -> f a -> f b

instance Functor Tree where
    fmap = mapTree

instance Functor [] where
    fmap = map

instance Functor Maybe where
    fmap _ Nothing = Nothing
    fmap f (Just x) = Just (f x)



: 